In [13]:
print("Hello World!")

Hello World!


In [1]:
import os
import torch
import torch.nn as nn
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from sklearn.metrics import accuracy_score
import numpy as np

# Setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
data_dir = "D:\\MiniProject\\AnimalWebApp\\DataSet2" 

# Data Transforms (with augmentation for training)
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
])


# Load Dataset
dataset = datasets.ImageFolder(data_dir, transform=transform)
data_loader = DataLoader(dataset, batch_size=4, shuffle=True)

# Class names
class_names = dataset.classes
print(f"Classes: {class_names}")

# Load Pretrained ResNet50
model = models.resnet50(pretrained=True)

# Freeze all layers
for param in model.parameters():
    param.requires_grad = False

# Replace final layer for binary classification
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)
model = model.to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.fc.parameters(), lr=0.001)

# Training Loop
num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    all_preds = []
    all_labels = []

    for images, labels in data_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

    acc = accuracy_score(all_labels, all_preds)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss:.4f}, Accuracy: {acc:.4f}")

# Save model
torch.save(model.state_dict(), "cats_vs_dogs_resnet50.pth")
print("Model saved!")

Classes: ['Cat', 'Dog']


d:\MiniProject\AnimalWebApp\env\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\MiniProject\AnimalWebApp\env\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/20, Loss: 95.9710, Accuracy: 0.7638
Epoch 2/20, Loss: 76.6457, Accuracy: 0.8225
Epoch 3/20, Loss: 102.4051, Accuracy: 0.7800
Epoch 4/20, Loss: 73.5011, Accuracy: 0.8413
Epoch 5/20, Loss: 72.7605, Accuracy: 0.8488
Epoch 6/20, Loss: 71.8642, Accuracy: 0.8500
Epoch 7/20, Loss: 67.9553, Accuracy: 0.8462
Epoch 8/20, Loss: 74.3272, Accuracy: 0.8400
Epoch 9/20, Loss: 74.6709, Accuracy: 0.8512
Epoch 10/20, Loss: 76.7949, Accuracy: 0.8350
Epoch 11/20, Loss: 82.4359, Accuracy: 0.8375
Epoch 12/20, Loss: 82.2886, Accuracy: 0.8225
Epoch 13/20, Loss: 100.8440, Accuracy: 0.8113
Epoch 14/20, Loss: 86.1081, Accuracy: 0.8425
Epoch 15/20, Loss: 56.0216, Accuracy: 0.8938
Epoch 16/20, Loss: 79.8600, Accuracy: 0.8462
Epoch 17/20, Loss: 111.4603, Accuracy: 0.8087
Epoch 18/20, Loss: 73.4420, Accuracy: 0.8650
Epoch 19/20, Loss: 77.2672, Accuracy: 0.8512
Epoch 20/20, Loss: 84.7469, Accuracy: 0.8337
Model saved!


In [5]:
from PIL import Image

# Load model
model = models.resnet50(pretrained=False)
model.fc = nn.Linear(model.fc.in_features, 2)
model.load_state_dict(torch.load("cats_vs_dogs_resnet50.pth"))
model = model.to(device)
model.eval()

# Prediction function
def predict_image(img_path):
    image = Image.open(img_path).convert('RGB')
    image = transform(image).unsqueeze(0).to(device)
    output = model(image)
    _, predicted = torch.max(output, 1)
    return class_names[predicted.item()]

# Example usage
print(predict_image("D:\\MiniProject\\AnimalWebApp\\DataSet\\Dog\\401.jpg"))

Dog
